In [1]:
import joblib
import tqdm
import numpy as np
import fitsio

import proplot as pplt

%matplotlib inline

In [6]:
def grid_file(*, fname, ngrid):
    d = fitsio.read(fname)

    dgrid = 1e4/ngrid
    xind = np.floor(d["x"] / dgrid)
    yind = np.floor(d["y"] / dgrid)
    gind = yind * ngrid + xind

    msk = (
        ((d["mask_flags"] & (~16)) == 0)
        & 
        (d["gauss_flags"] == 0)
        &
        (d["gauss_psf_flags"] == 0)
        & 
        (d["gauss_obj_flags"] == 0)
        &
        (d["psfrec_flags"] == 0)
        & 
        (d["gauss_T_ratio"] > 0.5)
        & (d["gauss_s2n"] > 10)
    )

    vals = []

    ugind = np.unique(gind)
    for _gind in range(ngrid*ngrid):
        gmsk = msk & (_gind == gind)
        if np.any(gmsk):
            sval = []
            for shear in ["noshear", "1p", "1m"]:
                sgmsk = gmsk & (d["mdet_step"] == shear)
                if np.any(sgmsk):
                    sval.append(np.mean(d["gauss_g_1"][sgmsk]))
                    sval.append(np.sum(sgmsk))
                else:
                    sval.append(np.nan)
                    sval.append(np.nan)
            vals.append(tuple(sval + [_gind]))
        else:
            vals.append(tuple([np.nan] * 6 + [_gind]))
    
    return np.array(vals, dtype=[("g1", "f8"), ("ng1", "f8"), ("g1p", "f8"), ("ng1p", "f8"), ("g1m", "f8"), ("ng1m", "f8"), ("grid_ind", "i4")])
            

def grid_file_pair(*, fplus, fminus, ngrid):
    dp = grid_file(fname=fplus, ngrid=ngrid)
    dm = grid_file(fname=fminus, ngrid=ngrid)

    assert np.all(dp["grid_ind"] == dm["grid_ind"])

    dt = []
    for tail in ["_p", "_m"]:
        for name in dp.dtype.names:
            if name != "grid_ind":
                dt.append((name + tail, "f8"))
    dt.append(("grid_ind", "i4"))
    d = np.zeros(ngrid * ngrid, dtype=dt)
    for _d, tail in [(dp, "_p"), (dm, "_m")]:
        for name in dp.dtype.names:
            if name != "grid_ind":
                d[name + tail] = _d[name]
    d["grid_ind"] = dp["grid_ind"]
    
    return d

In [7]:
def compute_shear_pair(d):
    g1_p = np.nansum(d["g1_p"] * d["ng1_p"]) / np.nansum(d["ng1_p"])
    g1p_p = np.nansum(d["g1p_p"] * d["ng1p_p"]) / np.nansum(d["ng1p_p"])
    g1m_p = np.nansum(d["g1m_p"] * d["ng1m_p"]) / np.nansum(d["ng1m_p"])
    R11_p = (g1p_p - g1m_p) / 0.02

    g1_m = np.nansum(d["g1_m"] * d["ng1_m"]) / np.nansum(d["ng1_m"])
    g1p_m = np.nansum(d["g1p_m"] * d["ng1p_m"]) / np.nansum(d["ng1p_m"])
    g1m_m = np.nansum(d["g1m_m"] * d["ng1m_m"]) / np.nansum(d["ng1m_m"])
    R11_m = (g1p_m - g1m_m) / 0.02

    return (g1_p - g1_m) / (R11_p + R11_m)

In [8]:
import glob

pfiles = sorted(glob.glob("sim_outputs_plus_23_*/*/*/metadetect/*_mdetcat_part*.fits"))
mfiles = sorted(glob.glob("sim_outputs_minus_23_*/*/*/metadetect/*_mdetcat_part*.fits"))

In [9]:
pmap = {pfile.split("/")[0]: pfile for pfile in pfiles}
mmap = {mfile.split("/")[0]: mfile for mfile in mfiles}

pairs = {}
for pk, pv in pmap.items():
    mk = pk.replace("_plus_", "_minus_")
    if mk in mmap:
        pairs[(pk, mk)] = (pv, mmap[mk])

In [10]:
len(pairs)

50

In [11]:
import joblib

jobs = [
    joblib.delayed(grid_file_pair)(fplus=pfile, fminus=mfile, ngrid=10)
    for pfile, mfile in pairs.values()
]

with joblib.Parallel(n_jobs=8, backend="loky", verbose=100) as par:
    d = par(jobs)

d = np.concatenate(d, axis=0)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:   39.3s
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   40.9s
[Parallel(n_jobs=8)]: Done   3 tasks      | elapsed:   41.2s
[Parallel(n_jobs=8)]: Done   4 tasks      | elapsed:   41.7s
[Parallel(n_jobs=8)]: Done   5 tasks      | elapsed:   42.1s
[Parallel(n_jobs=8)]: Done   6 tasks      | elapsed:   42.5s
[Parallel(n_jobs=8)]: Done   7 tasks      | elapsed:   43.0s
[Parallel(n_jobs=8)]: Done   8 tasks      | elapsed:   43.2s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:  1.1min
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:  1.2min
[Parallel(n_jobs=8)]: Done  11 tasks      | elapsed:  1.2min
[Parallel(n_jobs=8)]: Done  12 tasks      | elapsed:  1.2min
[Parallel(n_jobs=8)]: Done  13 tasks      | elapsed:  1.2min
[Parallel(n_jobs=8)]: Done  14 tasks      | elapsed:  1.2min
[Parallel(n_jobs=8)]: Done  15 tasks      | elapsed:  1.2min
[Parallel(

In [12]:
import tqdm

ns = 1000
seed = 10
rng = np.random.RandomState(seed=seed)

mn = compute_shear_pair(d)/0.02-1
vals = []
for i in tqdm.trange(ns, ncols=80):
    rind = rng.choice(d.shape[0], size=d.shape[0], replace=True)
    vals.append(compute_shear_pair(d[rind])/0.02-1)

print("m = %0.3f +/- %0.3f [1e-3, 3-sigma]" % (mn/1e-3, np.std(vals)*3/1e-3))

100%|██████████████████████████████████████| 1000/1000 [00:01<00:00, 566.23it/s]


m = -4.081 +/- 7.165 [1e-3, 3-sigma]


In [ ]:
!tail -f log23_*